In [ ]:
import os

os.makedirs("src", exist_ok=True)
os.makedirs("input", exist_ok=True)
os.makedirs("output", exist_ok=True)
open("src/__init__.py", "w").close()  # Empty __init__.py

In [ ]:
%%writefile src/symbol_table.py
class SymbolTable:
  """Creating symbol table for assembler"""
  def __init__(self):
      self.table = {
          'R0': 0,
          'R1': 1,
          'R2': 2,
          'R3': 3,
          'R4': 4,
          'R5': 5,
          'R6': 6,
          'R7': 7,
          'R8': 8,
          'R9': 9,
          'R10': 10,
          'R11': 11,
          'R12': 12,
          'R13': 13,
          'R14': 14,
          'R15': 15,
          'SP': 0,
          'LCL': 1,
          'ARG': 2,
          'THIS': 3,
          'THAT': 4,
          'SCREEN': 16384,
          'KBD': 24576}
      self.next_var_address = 16

  def add_entry(self, symbol, address):
      self.table[symbol] = address

  def contains(self, symbol):
      return symbol in self.table

  def get_address(self, symbol):
    return self.table[symbol]


Overwriting src/symbol_table.py


In [ ]:
%%writefile src/parser.py
class Parser:
  """Parser for assembler"""
  def __init__(self, input_file):
    self.input_file = input_file
    with open(input_file, "r") as file:
      self.lines = [line.split('//')[0].strip() for line in file if not line.strip().startswith('//') and line.strip()]
    self.current_line = 0
    self.file_path = input_file

  def has_more_lines(self):
    return self.current_line < len(self.lines)

  def advance(self):
    if self.has_more_lines():
      self.current_line += 1

  def instruction_type(self):
    line = self.lines[self.current_line]
    if line.startswith('@'):
      return 'A_INSTRUCTION'
    elif line.startswith('('):
      return 'L_INSTRUCTION'
    else:
      return 'C_INSTRUCTION'

  def symbol(self):
    line = self.lines[self.current_line]
    if self.instruction_type() == 'A_INSTRUCTION':
      return line[1:]
    elif self.instruction_type() == 'L_INSTRUCTION':
      return line.strip('()')

  def dest(self):
    if self.instruction_type() == 'C_INSTRUCTION':
      line = self.lines[self.current_line]
      return line.split('=')[0].strip() if '=' in line else ''

  def comp(self):
    if self.instruction_type() == 'C_INSTRUCTION':
      line = self.lines[self.current_line]
      return line.split('=')[-1].strip().split(';')[0].strip()

  def jump(self):
    if self.instruction_type() == 'C_INSTRUCTION':
      line = self.lines[self.current_line]
      return line.split(';')[1].strip() if ';' in line else ''



'''
parser = Parser("input/Max.asm")
for line in parser.lines:
  print(line)
  print(f"Instruction type: {parser.instruction_type()}")
  print(f"Symbol: {parser.symbol()}")
  print(f"Dest: {parser.dest()}")
  print(f"Comp: {parser.comp()}")
  print(f"Jump: {parser.jump()}")
  parser.advance()
'''


Overwriting src/parser.py


In [ ]:
%%writefile src/code.py
class Code:
  def __init__(self):
    #self.dest_table = {'': '000', 'M': '001', 'D': '010', 'DM': '011', 'A': '100', 'AM': '101', 'AD': '110', 'ADM': '111'}
    self.comp_table = {'0': '0101010', '1': '0111111', '-1':'0111010', 'D':'0001100', 'A':'0110000', '!D':'0001101','!A':'0110001','-D':'0001111', '-A': '0110011','D+1': '0011111', 'A+1': '0110111', 'D-1':'0001110', 'A-1':'0110010','D+A':'0000010','D-A':'0010011','A-D':'0000111','D&A':'0000000','D|A':'0010101', 'M':'1110000', '!M':'1110001','-M':'1110011','M+1':'1110111','M-1':'1110010','D+M':'1000010','D-M':'1010011','M-D':'1000111','D&M':'1000000','D|M':'1010101'}
    self.jump_table = {'': '000', 'JGT': '001', 'JEQ': '010', 'JGE': '011', 'JLT': '100', 'JNE': '101', 'JLE': '110', 'JMP': '111'}

  def dest(self, mnemonic):
    l_value = list(mnemonic)
    dest_bin = ''
    if 'A' in l_value:
      dest_bin += '1'
    else:
      dest_bin += '0'
    if 'D' in l_value:
      dest_bin += '1'
    else:
      dest_bin += '0'
    if 'M' in l_value:
      dest_bin += '1'
    else:
      dest_bin += '0'
    return dest_bin

  def comp(self, mnemonic):
    return self.comp_table.get(mnemonic, '000')

  def jump(self, mnemonic):
    return self.jump_table.get(mnemonic, '000')


#code = Code()
#print(code.dest('DA'))
#print(code.comp('M'))
#print(code.jump('JGT'))


Overwriting src/code.py


In [ ]:
a = 'md'
l_value = list(a)
print(l_value)
if 'A' in l_value:
  print('a')

['m', 'd']


In [ ]:
from src.parser import Parser
from src.symbol_table import SymbolTable
from src.code import Code

class Assembler:
  def __init__(self, input_file, output_file):
    self.input_file = input_file
    self.output_file = output_file
    self.parser = Parser(input_file)
    self.symbol_table = SymbolTable()
    self.code = Code()

  def first_pass(self):
    """ """
    rom_address = 0
    while self.parser.has_more_lines():
      instruction_type = self.parser.instruction_type()
      if instruction_type == 'A_INSTRUCTION' or instruction_type == 'C_INSTRUCTION':
        rom_address += 1
      elif instruction_type == 'L_INSTRUCTION':
        symbol = self.parser.symbol()
        self.symbol_table.add_entry(symbol, rom_address)
      self.parser.advance()

  def second_pass(self):
    """ """
    self.parser = Parser(self.input_file)
    with open(self.output_file, 'w') as out:
      while self.parser.has_more_lines():
        instruction_type = self.parser.instruction_type()
        if instruction_type == 'A_INSTRUCTION':
          symbol = self.parser.symbol()
          if symbol.isdigit():
            address = int(symbol)
          else:
            if not self.symbol_table.contains(symbol):
              self.symbol_table.add_entry(symbol, self.symbol_table.next_var_address)
              self.symbol_table.next_var_address += 1
            address = self.symbol_table.get_address(symbol)
          out.write(f"{address:016b}\n")
        elif instruction_type == 'C_INSTRUCTION':
          jump_part = self.parser.jump()
          dest_part = self.parser.dest()
          comp_part = self.parser.comp()
          jump_bits = self.code.jump(jump_part)
          dest_bits = self.code.dest(dest_part)
          comp_bits = self.code.comp(comp_part)
          out.write(f"111{comp_bits}{dest_bits}{jump_bits}\n")
        self.parser.advance()


assembler = Assembler('input/Pong.asm', 'output/Pong.hack')

assembler.first_pass()
parser = Parser(assembler.input_file)
for line in parser.lines:
  print(line)
assembler.second_pass()
with open(assembler.output_file, 'r') as f:
  for line in f:
    print(line)



Streaming output truncated to the last 5000 lines.
0000000001011111

1110101010000111

0000000000000000

1111110010101000

1111110000010000

1110110010100000

1111000010001000

0000000000000101

1110110000010000

0000000000000000

1111110111101000

1110110010100000

1110001100001000

0000000000000000

1111110010101000

1111110000010000

1110110010100000

1111000010001000

0000000000000000

1111110010101000

1111110000010000

0000000000000001

1111110111100000

1110110111100000

1110001100001000

0000000000000001

1111110111100000

1111110000010000

0000000000000000

1111110111101000

1110110010100000

1110001100001000

0000000000000000

1111110111001000

1111110010100000

1110111111001000

0000000000000000

1111110010101000

1111110000010000

1110110010100000

1111000111001000

0000000000000000

1111110010101000

1111110000010000

0000000000000001

1111110111100000

1110001100001000

0000000000000001

1111110000100000

1111110000010000

0000000000000000

1111110111101000

1110110010100